# Self-adaptive Evolution Strategy (SAES)

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\E}{\mathbb{E}}$

$\newcommand{\vs}[1]{\boldsymbol{#1}}$
$\newcommand{\ms}[1]{\boldsymbol{#1}}$

## Algorithm

---

Self-adaptive Evolution Strategy (SAES) with revaluations. ${\cal{N}}$ denotes some independent standard Gaussian random variable, with dimension as required in equations above.

---

**Parameters:**
$K>0$,
$\zeta\geq 0$,
$\lambda>\mu>0$
and a dimension $d>0$.

**Input:**
an initial parent population $x_{1,i} \in \R^d$
and an initial $\sigma_{1,i} = 1$ with $i \in \{1, \dots, \mu \}$.

---

$n\leftarrow 1$

**while** (stop condition) **do**

$\quad$ Generate $\lambda$ individuals $i_j$ independently with $j \in \{ 1, \dots, \lambda \}$ using
    
\begin{eqnarray}
    \sigma_j & = & \sigma_{n, mod(j-1, \mu) + 1} \times \exp\left( \frac{1}{2d} \cal{N} \right) \\
    i_j      & = & \vs{x}_{n, mod(j-1, \mu) + 1} + \sigma_j \cal{N}.
\end{eqnarray}
    
$\quad$ Evaluate each of them $\lceil Kn^\zeta \rceil$ times and average their fitness values

$\quad$ Define $j_1, \dots, j_{\lambda}$ so that

$$
\E_{\lceil Kn^\zeta \rceil}[f(i_{j_1})]\leq \E_{\lceil Kn^\zeta \rceil}[f(i_{j_2})] \leq \dots \leq \E_{\lceil Kn^\zeta \rceil}[f(i_{j_{\lambda}})]
$$

$\quad$ where $\E_m$ denotes a sample average over $m$ resamplings.

$\quad$ Update: compute $\vs{x}_{n+1, k}$ and $\sigma_{n+1, k}$ using

\begin{eqnarray}
\sigma_{n+1,k}   &=& \sigma_{j_{k}}, \quad k \in \{1, \dots, \mu\}\\
{\vs{x}_{n+1,k}} &=& i_{j_{k}},      \quad k \in \{1, \dots, \mu\}
\end{eqnarray}

$\quad$ $n\leftarrow n+1$

**end while**

## A Python inplementation (draft)

* http://www.scholarpedia.org/article/Evolution_strategies
* https://homepages.fhv.at/hgb/downloads/mu_mu_I_lambda-ES.oct

In [ ]:
"""Individual class

- x: the individual's value
- sigma: the individual's sigma
"""
class Individual():
    def __init__(self, x, sigma, y):
        self.x = x
        self.sigma = sigma
        self.y = y           # cost or reward

    def __str__(self):
        return "{0} {1} {2}".format(self.x, self.sigma, self.y)
    

mu = 3                # number of parents
lambda_ = 12          # number of offspring
x_init = None         # initial parent vector
sigma_init = 1        # initial global mutation strength sigma
sigma_min = 1e-5      # ES stops when sigma is smaller than sigma_min
num_evals_func = None # How many times noisy objective functions should be called for each evaluation (considere the average value of these calls)

log.data["x"] = []
log.data["sigma"] = []
log.data["y"] = []


def select_individuals(pop):
    """Sort the population according to the individuals' fitnesses.
    
    - pop: a list of Individual objects
    """
    pop.sort(key=lambda indiv: indiv.y, reverse=False)
    return pop[:mu]


def recombine_individuals(parents):
    """Perform intermediate (multi-)recombination.
    
    - parents: a list of Individual objects
    """
    parents_y = np.array([indiv.x for indiv in parents])
    parents_sigma = np.array([indiv.sigma for indiv in parents])
    recombinant = Individual(parents_y.mean(axis=0), parents_sigma.mean(), 0) # TODO
    return recombinant


def optimize(objective_function, num_gen=50):

    #if x_init is None:
    #    x_init = np.random.random(objective_function.ndim)
    #    x_init -= (objective_function.domain_min + objective_function.domain_max) / 2.  # TODO
    #    x_init *= (objective_function.domain_max - objective_function.domain_min)       # TODO
    #else:
    assert x_init.ndim == 1
    assert x_init.shape[0] == objective_function.ndim

    # Initialization
    n = x_init.shape[0]         # determine search space dimensionality n
    tau = 1. / math.sqrt(2.*n)       # self-adaptation learning rate

    # Initializing individual population
    y = objective_function(x_init)
    parent_pop = [Individual(x_init, sigma_init, y) for i in range(mu)]

    gen_index = 0

    # Evolution loop of the (mu/mu_I, lambda)-sigma-SA-ES
    while parent_pop[0].sigma > sigma_min and gen_index < num_gen:
        offspring_pop = []
        recombinant = recombine_individuals(parent_pop) # TODO: BUG ? this statement may be in the next line
        for offspring_index in range(1, lambda_):
            offspring_sigma = recombinant.sigma * math.exp(tau * random.normalvariate(0,1))
            offspring_x = recombinant.x + offspring_sigma * np.random.normal(size=n)

            if num_evals_func is None:
                # If the objective function is deterministic
                offspring_y = objective_function(offspring_x)
            else:
                # If the objective function is stochastic
                # TODO: move this in function or in optimizer (?) class so that it is available for all optimiser implementations...
                num_evals = num_evals_func(gen_index)
                offspring_y_list = np.zeros(num_evals)
                for eval_index in range(num_evals):
                    offspring_y_list[eval_index] = float(objective_function(offspring_x))
                offspring_y = np.mean(offspring_y_list)
                # TODO: generate the confidence bounds of offspring_y and plot it

            offspring = Individual(offspring_x, offspring_sigma, offspring_y)
            offspring_pop.append(offspring)
        parent_pop = select_individuals(offspring_pop)
        #parent_pop = select_individuals(parent_pop + offspring_pop)

        gen_index += 1

        log.data["x"].append(parent_pop[0].x)  # TODO use a "log" object instead
        log.data["y"].append(parent_pop[0].y)  # TODO
        print(parent_pop[0])

    plotSamples(np.array(log.data["x"]), np.array(log.data["y"]), objective_function=objective_function)
    plotCosts(np.array(log.data["y"]))

    return parent_pop[0].x

## "Standalone" version (pyai/standalone_implementations/saes/saes.py)

In [ ]:
"""
This is a simple Python implementation of the (mu/mu_I, lambda)-sigmaSA-ES
as discussed in
http://www.scholarpedia.org/article/Evolution_Strategies
Based on
https://homepages.fhv.at/hgb/downloads/mu_mu_I_lambda-ES.oct
"""

import math
import numpy as np
import matplotlib.pyplot as plt
import random


MU = 3                    # number of parents
LAMBDA = 12               # number of offspring

X_INIT = np.ones(1)       # initial parent vector
#X_INIT = np.ones(2)       # initial parent vector
SIGMA_INIT = 1            # initial global mutation strength sigma

SIGMA_MIN = 1e-10         # ES stops when sigma is smaller than sigma_min

###########################################################

# Function to be optimized (sphere test function as an example)
# individual: the individual to evaluate
def sphere_function(indiv):
    x = indiv.x
    assert x.shape == X_INIT.shape, x
    y = np.dot(x, x)
    return y

def sin1(indiv):
    x = indiv.x
    assert x.shape == (1,), x
    x = np.absolute(x)
    y = np.sin(2 * 2 * np.pi * x) * np.exp(-5 * x)
    return y

def sin2(indiv):
    x = indiv.x
    assert x.shape == (1,), x
    y = np.sin(2 * 2 * np.pi * x) * 1/np.sqrt(2*np.pi) * np.exp(-(x**2)/2)
    return y

from matplotlib.finance import quotes_historical_yahoo
import datetime

date1 = datetime.date( 1995, 1, 1 )
date2 = datetime.date( 2004, 4, 12 )
quotes = quotes_historical_yahoo('INTC', date1, date2)
yahoo_data = [-q[1] for q in quotes]

def yahoo(indiv):
    x = indiv.x
    assert x.shape == (1,), x

    y = 0
    if 0 <= x < len(yahoo_data):
        y = yahoo_data[int(x)]
    return y

#fitness = sphere_function
fitness = sin1
#fitness = sin2
#fitness = yahoo

###########################################################

# The individual class
# x: the individual's value
# sigma: the individual's sigma
class Individual():
    def __init__(self, x, sigma):
        self.x = x
        self.sigma = sigma
        self.cost = fitness(self)

    def __str__(self):
        return "{0} {1} {2}".format(self.x, self.sigma, self.cost)

###########################################################

# This sorts the population according to the individuals' fitnesses
# pop: a list of Individual objects
def select_individuals(pop):
    pop.sort(key=lambda indiv: indiv.cost, reverse=False)
    return pop[:MU]

# This performs intermediate (multi-) recombination
# parents: a list of Individual objects
def recombine_individuals(parents):
    parents_y = np.array([indiv.x for indiv in parents])
    parents_sigma = np.array([indiv.sigma for indiv in parents])
    recombinant = Individual(parents_y.mean(axis=0), parents_sigma.mean())
    return recombinant

###########################################################

def main():

    # Initialization
    n = X_INIT.shape[0]         # determine search space dimensionality n
    tau = 1. / math.sqrt(2.*n)  # self-adaptation learning rate

    # Initializing individual population
    parent_pop = [Individual(X_INIT, SIGMA_INIT) for i in range(MU)]

    # Evolution loop of the (mu/mu_I, lambda)-sigma-SA-ES
    while parent_pop[0].sigma > SIGMA_MIN:
        offspring_pop = []
        recombinant = recombine_individuals(parent_pop) # TODO: BUG ? this statement may be in the next line
        for i in range(1, LAMBDA):
            offspring_sigma = recombinant.sigma * math.exp(tau * random.normalvariate(0,1))
            offspring_y = recombinant.x + offspring_sigma * np.random.normal(size=n)
            offspring = Individual(offspring_y, offspring_sigma)
            offspring_pop.append(offspring)
        parent_pop = select_individuals(offspring_pop)
        print(parent_pop[0])

# Remark: Final approximation of the optimizer is in "parent_pop[0].x"
#         corresponding fitness is in "parent_pop[0].cost" and the final
#         mutation strength is in "parent_pop[0].sigma"

if __name__ == "__main__":
    main()